In [1]:
import xarray as xr
import numpy as np
import earthkit.data as ekd
from earthkit.transforms.aggregate import spatial
from dhis2eo.stac import WorldPopSTAC

In [2]:
pm_file = "./data/pm_final_srilanka_linearp.nc"
pm_ds = xr.open_dataset(pm_file, chunks={"time": 100})
pm_data = pm_ds["__xarray_dataarray_variable__"]

district_file = "./data/geoBoundaries-LKA-ADM2.geojson"
features = ekd.from_source("file", district_file)

wp = WorldPopSTAC()

years = np.unique(pm_data['time'].dt.year).astype(str)

for year in years:
  print(f"Calculating {year}") 
  pm_year = pm_data.sel(time=year) 
    
  pop_file = wp.get_item(
    collection='LKA',
    year=year,
    resolution='1km',
    project='Population'
  ).get_asset().download(out_dir='./data')
  pop_ds = xr.open_dataset(pop_file)
  pop_data = pop_ds['band_data'].rename({"x": "lon", "y": "lat"})
  pop_aligned = pop_data.interp(lon=pm_year.lon, lat=pm_year.lat, method="nearest")

  pm_weighted = pm_year * pop_aligned

  agg_num = spatial.reduce(pm_weighted, features, how="sum", mask_dim="shapeName")
  agg_den = spatial.reduce(pop_aligned, features, how="sum", mask_dim="shapeName")

  pw = agg_num / agg_den
  pw.name = "pm25_popweighted"

  agg_df = pw.to_dataframe().reset_index()
  print(agg_df)

Calculating 2020


/tmp/ipykernel_88113/4071827174.py:28: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_num = spatial.reduce(pm_weighted, features, how="sum", mask_dim="shapeName")
/tmp/ipykernel_88113/4071827174.py:29: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_den = spatial.reduce(pop_aligned, features, how="sum", mask_dim="shapeName")


                 shapeName       time  band  spatial_ref  pm25_popweighted
0          Jaffna District 2020-03-01     1            0         43.995458
1          Jaffna District 2020-03-02     1            0         64.471118
2          Jaffna District 2020-03-03     1            0         83.498582
3          Jaffna District 2020-03-04     1            0         50.151084
4          Jaffna District 2020-03-05     1            0         38.137212
...                    ...        ...   ...          ...               ...
7645  Trincomalee District 2020-12-27     1            0         13.577054
7646  Trincomalee District 2020-12-28     1            0         10.171014
7647  Trincomalee District 2020-12-29     1            0          7.533393
7648  Trincomalee District 2020-12-30     1            0         11.120927
7649  Trincomalee District 2020-12-31     1            0         10.560595

[7650 rows x 5 columns]
Calculating 2021


/tmp/ipykernel_88113/4071827174.py:28: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_num = spatial.reduce(pm_weighted, features, how="sum", mask_dim="shapeName")
/tmp/ipykernel_88113/4071827174.py:29: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_den = spatial.reduce(pop_aligned, features, how="sum", mask_dim="shapeName")


                 shapeName       time  band  spatial_ref  pm25_popweighted
0          Jaffna District 2021-01-01     1            0         50.440638
1          Jaffna District 2021-01-02     1            0         38.330146
2          Jaffna District 2021-01-03     1            0         26.815412
3          Jaffna District 2021-01-04     1            0         20.779433
4          Jaffna District 2021-01-05     1            0         18.650441
...                    ...        ...   ...          ...               ...
9120  Trincomalee District 2021-12-27     1            0         30.052926
9121  Trincomalee District 2021-12-28     1            0         30.905051
9122  Trincomalee District 2021-12-29     1            0         41.566963
9123  Trincomalee District 2021-12-30     1            0         35.758512
9124  Trincomalee District 2021-12-31     1            0         27.145606

[9125 rows x 5 columns]
Calculating 2022


/tmp/ipykernel_88113/4071827174.py:28: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_num = spatial.reduce(pm_weighted, features, how="sum", mask_dim="shapeName")
/tmp/ipykernel_88113/4071827174.py:29: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_den = spatial.reduce(pop_aligned, features, how="sum", mask_dim="shapeName")


                 shapeName       time  band  spatial_ref  pm25_popweighted
0          Jaffna District 2022-01-01     1            0         21.118635
1          Jaffna District 2022-01-02     1            0         18.038370
2          Jaffna District 2022-01-03     1            0         22.697283
3          Jaffna District 2022-01-04     1            0         28.645449
4          Jaffna District 2022-01-05     1            0         14.555859
...                    ...        ...   ...          ...               ...
9120  Trincomalee District 2022-12-27     1            0         19.230062
9121  Trincomalee District 2022-12-28     1            0         21.278467
9122  Trincomalee District 2022-12-29     1            0         23.543924
9123  Trincomalee District 2022-12-30     1            0         21.842143
9124  Trincomalee District 2022-12-31     1            0         22.629363

[9125 rows x 5 columns]
Calculating 2023


/tmp/ipykernel_88113/4071827174.py:28: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_num = spatial.reduce(pm_weighted, features, how="sum", mask_dim="shapeName")


                 shapeName       time  band  spatial_ref  pm25_popweighted
0          Jaffna District 2023-01-01     1            0         52.004230
1          Jaffna District 2023-01-02     1            0         42.220692
2          Jaffna District 2023-01-03     1            0         49.236588
3          Jaffna District 2023-01-04     1            0         56.594455
4          Jaffna District 2023-01-05     1            0         49.203899
...                    ...        ...   ...          ...               ...
9120  Trincomalee District 2023-12-27     1            0         18.800796
9121  Trincomalee District 2023-12-28     1            0         22.124807
9122  Trincomalee District 2023-12-29     1            0         18.672990
9123  Trincomalee District 2023-12-30     1            0         26.025079
9124  Trincomalee District 2023-12-31     1            0         24.395194

[9125 rows x 5 columns]


/tmp/ipykernel_88113/4071827174.py:29: DeprecationWarning: The function 'reduce' from the legacy aggregate module is deprecated and will be removed in version 2.X of earthkit.transforms. Use 'earthkit.transforms.spatial.reduce' instead.
  agg_den = spatial.reduce(pop_aligned, features, how="sum", mask_dim="shapeName")
